**Course**: Data Visualization (Prof. Dr. Heike Leitte, Luisa Vollmer, RPTU Kaiserslautern),   **Name**: Faris Abu Ali,   **Date**: 08.01.2025

<div class="alert alert-info">

    
# Assignment 4 - Principal Component Analysis (PCA)
</div>


### Outline and goals

In the fourth assignment, we will explore a cars dataset using principal component analysis. Each car is described by 25 variables. The dataset contains six types of cars and we would like to understand how they are different. The goal of this assignment is that you are able to:
- explore high-dimensional data with many variables
- compute and interpret PCA
- charactize groups in the PCA plot
- find patterns and outliers in data with many variables

<div class="alert alert-danger">

**Important**: While no points will be awarded for typing the correct answers in the notebooks, it is highly advised to solve the tasks thoroughly. They are designed to be encouraging and provide you with valuable learnings for the exam, understanding of the methods and practical coding.
</div>

<div class="alert alert-success">
    
All tasks in this notebook are marked in green.
</div>

In [254]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from bokeh.models import ColumnDataSource, ColorBar, LinearColorMapper, CategoricalColorMapper
from bokeh.models import Arrow, NormalHead, LabelSet, Label
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Category10, Category20, Viridis
from bokeh.transform import factor_cmap, linear_cmap
from bokeh.io import export_png
from bokeh.layouts import gridplot, row
from bokeh.core.properties import value
from bokeh.models.tickers import FixedTicker

from math import pi

output_notebook()

<div class="alert alert-info">
    
## 1. Load the 1993 cars dataset
</div>

First, we load the data and handle missing values. We ignore the variable LUGGAGE CAPACITY which has no information for vans and some missing values. We also drop two cars with missing values.

Description of variables:
- Manufacturer
- Model
- Type: Small, Sporty, Compact, Midsize, Large, Van
- Minimum Price (in \$1,000) - Price for basic version of this model
- Midrange Price (in \$1,000) - Average of Min and Max prices
- Maximum Price (in \$1,000) - Price for a premium version
- City MPG (miles per gallon by EPA rating)
- Highway MPG
- Air Bags standard 0 = none, 1 = driver only, 2 = driver & passenger
- Drive train type 0 = rear wheel drive 1 = front wheel drive 2 = all wheel drive
- Number of cylinders
- Engine size (liters)
- Horsepower (maximum)
- RPM (revs per minute at maximum horsepower)
- Engine revolutions per mile (in highest gear)
- Manual transmission available 0 = No, 1 = Yes
- Fuel tank capacity (gallons)
- Passenger capacity (persons)
- Length (inches)
- Wheelbase (inches)
- Width (inches)
- U-turn space (feet)
- Rear seat room (inches)
- Luggage capacity (cu. ft.)
- Weight (pounds)
- Domestic? 0 = non-U.S. manufacturer 1 = U.S. manufacturer

In [255]:
# load the data
cars = pd.read_csv( '93cars.dat.csv', sep='\s+', na_values='*')
print("size of input dataframe", cars.shape)


# substitute missing values
cars.drop(['LuggageCapacity'], axis=1, inplace=True)
cars.dropna(inplace=True)
cars.reset_index(inplace=True, drop=True)

print("size after removing NaN", cars.shape)
print("variables in dataframe:", list(cars))

size of input dataframe (93, 26)
size after removing NaN (91, 25)
variables in dataframe: ['Manufacturer', 'Model', 'Type', 'MinPrice', 'MidPrice', 'MaxPrice', 'CityMpg', 'HighwayMpg', 'AirBags', 'DriveTrainType', 'Cylinders', 'Engine', 'Horsepower', 'RPM', 'EngineRev', 'ManTrans', 'Tank', 'Passenger', 'Length', 'Wheelbase', 'Width', 'UTurn', 'RearSeatRoom', 'Weight', 'Domestic']


First render the entire data using seaborn and its scatterplot matrix function. This may take very long and you have a pdf-export in your folder. No need to run this code, unless you would like to see it in the notebook and try.

In [256]:
import seaborn as sns

# This could take a while
# sns_plot = sns.pairplot(cars, hue="Type")

# uncomment to export plot
#sns_plot.savefig("cars93_SPLOM.pdf") # SPLOM = Scatter Plot Matrix

<div class="alert alert-info">
    
## 2. Understand the variables
</div>

Each car is described by a long list of variables and analyzing them individually and pair-wise will take a long time. In this assignment we will first reduce the dimensionality of the dataset and then analyze it.

### Explore scatterplot matrix

<div class="alert alert-success">
    
To understand the problem take a look at the scatterplot matrix provided in your data folder ([cars93_SPLOM.pdf](cars93_SPLOM.pdf)). What can you tell about small cars? (blue points, color legend is on the right) (no answer required and don't spend more than a few minutes)
</div>

### Explore correlation matrix

Your first task is to find groups of variables that belong together and understand their connection.
Therefore, we compute [linear correlation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html) as provided by pandas. As bokeh requires the input as a linear array, we also linearize the matrix.

In [257]:
# compute Pearson correlation
corr = cars.corr(method='pearson', numeric_only=True)
# print("correlation matrix:\n", corr)
variables = list(corr)
# print("variables in correlation matrix:", variables)

# linearize the correlation matrix
lin_corr = pd.melt(corr.assign(index=corr.index), id_vars=['index'])
# print(lin_corr)
lin_corr['size'] = [abs(val)*17 for val in lin_corr.value] # multiply by 17 only for plotting purpose below. This makes the squares larger

corr
lin_corr
# lin_corr[lin_corr['index'] == 'MinPrice']

,index,variable,value,size
0,MinPrice,MinPrice,1.000000,17.000000
1,MidPrice,MinPrice,0.969497,16.481444
2,MaxPrice,MinPrice,0.904767,15.381047
3,CityMpg,MinPrice,-0.613667,10.432340
4,HighwayMpg,MinPrice,-0.574668,9.769355
...,...,...,...,...
479,Width,Domestic,0.428959,7.292298
480,UTurn,Domestic,0.480740,8.172585
481,RearSeatRoom,Domestic,0.103606,1.761307
482,Weight,Domestic,0.209338,3.558746


The following code renders a correlation matrix using bokeh. For each pair of variables, the correlation is represented by a colored square. Size and color encode the amount of linear correlation. Large red squares indicate strong positive correlation, large blue squares strong negative correlation.

In [258]:
p = figure(x_range=variables, y_range=variables, width=800, height=700,
           title="Correlation matrix")

p.scatter(marker='square', source=lin_corr, x='index', y='variable', size='size', color=linear_cmap(field_name='value', palette='RdYlBu7', low=-1, high=1))

color_bar = ColorBar(color_mapper=LinearColorMapper('RdYlBu5', low=-1, high=1), # because Pearson correlation factor has the range [-1, 1]
                    #  -1 ==> strongly Negativly Correlated.
                    #  +1 ==> strongly Positively Correlated.
                    #  0  ==> No Correlation
                     label_standoff=12, border_line_color=None, location=(0,0))
p.add_layout(color_bar, 'right')

p.xgrid.ticker = FixedTicker(ticks=list(range(1,len(corr))))
p.ygrid.ticker = FixedTicker(ticks=list(range(1,len(corr))))
p.xaxis.major_label_orientation = -pi/4 # to rotate the label instead of being horizontal
show(p)

<div class="alert alert-success">
    
**Find groups of variables that are strongly correlated** (positive or negative). Each group consists of variables that are pair-wise strongly correlated. The example below shows the group "Properties of the engine" which consists of horsepower, engine, cylinders.
    
**Tasks**
- Find three additional groups, name them, and list the variables that belong to them.
- Analyze if these high-level groups are correlated: Locate the positions in the matrix that encode the correlations between elements from two groups. Can you detect patterns?
- Validate your findings using the scatterplot matrix.
</div>

![](cars_corrMatrix.png)

<div class="alert alert-warning">

### Groups:
- **Engine properties:** horsepower, engine, cylinders
  - strongly positively correlated, reflecting the engine's properties.

<br/>

- **Vehicle Size:** Length, Width, Wheelbase, Weight.
  - strongly positively correlated, reflecting vehicle dimensions and mass.

<br/>

- **Fuel Efficiency:** CityMpg, HighwayMpg
  - strongly positively correlated, as they both measure fuel efficiency under different conditions.

<br/>

- **Price Metrics:** MinPrice, MidPrice, MaxPrice.
  - strongly positively correlated as they represent price ranges for the same vehicle.

<br/>

- **EngineRev & Cylinders:**
  - strongly negatively correlated.

<br/>

### Additional correlations between groups:
- **(Engine properties) + (Vehicle Size) --> POSITIVELY correlated**
  - Looking at the correlation matrix, we can notice that the two groups are also positively correlated to each other.
  👉 [engine-properties-and-vehicle-size](https://drive.google.com/file/d/1F4v2QO8fyaIbBGpXlauV00QDgmPzy5fZ/view?usp=sharing)

<br/>

- **(Fuel Efficiency) + (Price Metrics) --> NEGATIVELY correlated**
  - From the correlation matrix, I have noticed that these two groups have light blue squares in the areas of cross-intersection, which means they have negative correlation with each other with Pearson correlation coefficient of approximately 0.5.
  - Looking at the pdf file SPLOM (Scatter Plot Matrix) also verifies the finding:
  👉 [fuel-efficiency-and-price-metrics](https://drive.google.com/file/d/16DauCcD4TBmAf32lTuJqUL-F3f0DYTDf/view?usp=sharing)

<br/>

- **(Fuel Efficiency) + (Vehicle Size) --> NEGATIVELY correlated**
  - 👉 [fuel-efficiency-and-vehicle-size](https://drive.google.com/file/d/12n86LkjlZau2p4M5mff7pm4cpTcdhs8K/view?usp=drive_link)

<br/>

- **(Engine properties) + (Price Metrics) --> POSITIVELY correlated**
    - Better motorization (properties of the engine), means higher price.
</div>

<div class="alert alert-info">
    
## 3. Explained variance of PCA
</div>

Compute the [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) for the cars dataset

In [259]:
# only use numeric attributes and exclude the type which we would like to predict
var = ['MinPrice', 'MidPrice','MaxPrice', 'CityMpg', 'HighwayMpg', 'Cylinders',
       'Engine', 'Horsepower', 'RPM', 'EngineRev',
       'Tank', 'Passenger', 'Length', 'Width', 'UTurn', 'Weight']

# store standardized data in cars_std
cars_std = StandardScaler().fit_transform(cars[var]) # shape: (91, 16) -- 91 rows, 16 variables

# store PCA in variable pca
pca = PCA(n_components=len(var)).fit(cars_std)

In [260]:
var_exp = pca.explained_variance_ratio_*100 # a list of the variances for each variable (column) multiplied by 100 to be represented as percentage
# print(var_exp)
cum_var_exp = np.cumsum(var_exp) # cumulative sum of the variances. i.e. cum_var_exp[n] = \sum_{i=0}^{n} var_exp[i]
# print(cum_var_exp)
x = [f'PC{i+1}' for i in range(len(var))]

source = ColumnDataSource( dict(x=x, var_exp=var_exp, cum_var_exp=cum_var_exp) )

p = figure( width=520, height=400, toolbar_location=None, x_range=x, y_range=(-2,105),
            title="Explained variance of PCA of cars dataset")

p.vbar( source=source, x='x', top='var_exp', width=0.9, bottom=0, legend_label='Explained variance' )

p.scatter( x, cum_var_exp, color='orange', size=5, legend_label="Cumulative explained variance")
p.line( x, cum_var_exp, color='orange', line_width=2 )

p.legend.location = (235,155)
p.legend.border_line_color = None
p.xgrid.visible = False
p.yaxis.axis_label = "Explained variance in percent"

show(p)

<div class="alert alert-success">

**Questions**
- How much variance is explained by the first two principal component(s) roughly?
- How many components do you need to explain 90% of the variance in the data (roughly, use figure estimate)?
</div>

<div class="alert alert-warning">
<b>Answers</b>

- PC1 (≈ 64%) + PC2 (≈ 15%) ≈ 79% of the variance is explained by the first two principal components.

- We need the first 5 components to explain 90% of the variance in the data.  

</div>

<div class="alert alert-info">
    
## 4. Interpret the projection
</div>

In [261]:
# project the data and add the labels using the cars' type
pca_cars = pd.DataFrame( pca.transform(cars_std), columns=[f'PC{i+1}' for i in range(pca.n_components_)])
pca_cars['label'] = cars.Type # Small, Midsize, Compact, Van, Sporty

# Again, why did we do this? WHy not from the beginning include the `Type` column in the cars_std?
# Answer: Because `Type` is categorial (non-numeric) feature, and we wanted to include only numeric columns in the poc all PCA components
# to be numeric when we instantiated the PCA instance:
# > pca = PCA(n_components=len(var)).fit(cars_std)

# get the different classes in the Type variable
factors = sorted(pca_cars.label.unique()) # ['Compact', 'Large', 'Midsize', 'Small', 'Sporty', 'Van']

In [262]:
source = ColumnDataSource(pca_cars)

p = figure( width=600, height=600, y_range=(-4.5,4.8),
            title="Projection onto first two principal components")

p.scatter( source=source, x='PC1', y='PC2', size=9, legend_group='label',
          color=factor_cmap(field_name='label', palette=Category10[10], factors=factors))
p.xaxis.axis_label = 'PC1'
p.yaxis.axis_label = 'PC2'

show(p)

<div class="alert alert-success">

**Explain the axes**

The figure above shows the projected cars data. Explain the x- and y-axis. What can you tell about cars that are located on the left/right and what about cars at the top/bottom?
</div>

Hints:
- You cannot solve the problem using the current chart only. You need some additional technique as explained in lecture.
- The eigenvectors can be obtained through `pca.components_`
- A biplot is most helpful for the upcoming questions.

In [263]:
components = pca.components_ # 2D matrix of shape: (16, 16)

feature_names = var
# Rows represent principal components (PC1, PC2, ..., PC16).
# Columns represent the original features in the dataset.

# - The first row (components[0]) corresponds to PC1.
# - The second row (components[1]) corresponds to PC2.
# - The values in each row indicate the contribution (or "weight") of each original feature to the respective principal component.

pc1 = components[0]
pc2 = components[1]

# np.argsort returns the indices that would sort an array.
# np.abs(pc1)
# [::-1] reverses the array so that the indices are indices of the array sorted descending

pc1_idxs = np.argsort(np.abs(components[0]))[::-1]  # Sorted by magnitude descending
pc2_idxs = np.argsort(np.abs(components[1]))[::-1]

pc1_features = [feature_names[i] for i in pc1_idxs]
pc2_features = [feature_names[i] for i in pc2_idxs]

print("\nTop features for PC1:", pc1_features)
print("Top features for PC2:", pc2_features)

# Print PC contributions sorted:
print("\nPC1 contributions:")
print('----------------------------------------')
for feature, weight in zip(pc1_features, pc1[pc1_idxs]):
    print(f"{feature}: {weight:.4f}")

print("\nPC2 contributions:")
print('----------------------------------------')
for feature, weight in zip(pc2_features, pc2[pc2_idxs]):
    print(f"{feature}: {weight:.4f}")


Top features for PC1: ['Weight', 'Engine', 'Tank', 'Width', 'Cylinders', 'CityMpg', 'Length', 'Horsepower', 'HighwayMpg', 'UTurn', 'EngineRev', 'MinPrice', 'MidPrice', 'MaxPrice', 'Passenger', 'RPM']
Top features for PC2: ['RPM', 'MaxPrice', 'MidPrice', 'MinPrice', 'Passenger', 'Horsepower', 'UTurn', 'Width', 'EngineRev', 'Length', 'Engine', 'CityMpg', 'Weight', 'Tank', 'HighwayMpg', 'Cylinders']

PC1 contributions:
----------------------------------------
Weight: 0.2987
Engine: 0.2895
Tank: 0.2752
Width: 0.2746
Cylinders: 0.2716
CityMpg: -0.2716
Length: 0.2632
Horsepower: 0.2562
HighwayMpg: -0.2529
UTurn: 0.2486
EngineRev: -0.2475
MinPrice: 0.2395
MidPrice: 0.2290
MaxPrice: 0.2113
Passenger: 0.1803
RPM: -0.1391

PC2 contributions:
----------------------------------------
RPM: 0.4413
MaxPrice: 0.4369
MidPrice: 0.4196
MinPrice: 0.3743
Passenger: -0.3100
Horsepower: 0.2607
UTurn: -0.2084
Width: -0.1990
EngineRev: 0.1780
Length: -0.0899
Engine: -0.0818
CityMpg: 0.0393
Weight: -0.0379
Tan

**Principal Components (PCs):**

- PC1 and PC2 are not individual features; they are linear combinations of all original features.
- Each feature (e.g., RM, Weight) contributes to both PC1 and PC2, but the magnitude of their contributions (loadings) determines which PC they influence more.

In [264]:
from bokeh.layouts import row

source = ColumnDataSource(dict(
    pc1_features=pc1_features,
    pc2_features=pc2_features,

    pc1_scores=pc1[pc1_idxs],
    pc2_scores=pc2[pc2_idxs]
))

pc1_figure = figure(
    width=600,
    height=600,
    title="PC1 features contribution",
    toolbar_location=None,
    y_range=pc1_features,
)


pc1_figure.hbar(
    left=0,
    right='pc1_scores',
    y='pc1_features',
    source=source,
    height=0.5,
    legend_label='PC1'
)

pc2_figure = figure(
    width=600,
    height=600,
    title="PC2 features contribution",
    toolbar_location=None,
    y_range=pc2_features,
)

pc2_figure.hbar(
    left=0,
    right='pc2_scores',
    y='pc2_features',
    source=source,
    height=0.5,
    legend_label='PC2'
)



show(row(pc1_figure, pc2_figure))


**Magnitude:**
- The size of the loading indicates how much a feature contributes to that principal component (PC).
- Larger values (positive or negative) imply stronger influence.

**Sign:**
- The sign (positive or negative) shows the direction of the relationship between the feature and the principal component.
- A negative loading suggests the feature contributes in the opposite direction compared to features with positive loadings for that component.

In [265]:
import numpy as np
import pandas as pd
from bokeh.models import ColumnDataSource, Arrow, VeeHead
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Extract the corresponding vectors
arrows = [
    components[:, pc1_idxs[0]], # (x,y) coordinates for the arrow
    components[:, pc2_idxs[0]]
]
feature_labels = [
    feature_names[pc1_idxs[0]],
    feature_names[pc2_idxs[0]]
]
# feature_labels = [
#     ', '.join([feature_names[idx] for idx in pc1_idxs[:3]]),
#     ', '.join([feature_names[idx] for idx in pc2_idxs[:3]])
# ]

# Scale vectors for visualization
scaling_factor = 4
arrows = [vector * scaling_factor for vector in arrows]

p = figure(width=600, height=600, title="Biplot: Projection with Top Features")

# Scatter plot
p.scatter(
    source=pca_cars, x='PC1', y='PC2', size=9, legend_group='label',
    color=factor_cmap(field_name='label', palette=Category10[10], factors=factors)
)

# Add arrows for the top feature of PC1 and PC2
for arrow, label in zip(arrows, feature_labels):
    arrow_glyph = Arrow(end=VeeHead(size=10),
                        x_start=0, y_start=0,
                        x_end=arrow[0], y_end=arrow[1],
                        line_width=2)
    p.add_layout(arrow_glyph)
    # Annotate feature names
    p.text(x=[arrow[0]], y=[arrow[1]], text=[label], text_font_size="12pt", text_font_style='bold')

# Set axis labels
p.xaxis.axis_label = 'PC1'
p.yaxis.axis_label = 'PC2'

show(p)


From The Above Code Analysis:

- The dominant features for PC1: [`Weight`, `Engine`, `Tank`]
- The dominant features for PC2: [`RPM`, `MaxPrice`, `MidPrice`]

<br/>

**X-axis (PC1):** The dominant features contributing to PC1 are 'Weight', 'Engine', and 'Tank', which suggests that **this axis primarily represents the size and power of the car.**
- Cars on the left (PC1 < 0) tend to be smaller, lighter, and have smaller engines and fuel tanks compared to the average.
- In contrast, cars on the right (PC1 > 0) are larger, heavier, and have more powerful engines with greater tank capacity.

<br/>

**Y-Axis (PC2):** For the Y-axis (PC2): The dominant features for PC2 are 'RPM', 'MaxPrice', and 'MidPrice', indicating that **this axis is likely related to performance and cost.**
- Cars at the top (PC2 > 0) tend to have higher RPM, suggesting sportier performance, and are also more expensive, indicating luxury or premium vehicles.
- Cars at the bottom (PC2 < 0) are generally more affordable and have lower RPM, reflecting more economical or utilitarian models.

**Observations:**
- Cars at the bottom are cheaper than those at the top. (Price strongly contributes in PC2 the vertical axis).
- Midsize (green) cars vary strongly in their price. (Their vertical variance is large)
- Midsize (green) cars are in general the most expensive type of car
- Large (orange) cars have a lower weight than small (red) cars.

<div class="alert alert-success">

**Characterize small cars**

You already tried to characterize small cars using the scatterplot matrix. Now do the same again using the PCA-plot and the analyses you made about the axes. What can you say about small cars? (small cars are red in this plot!)
    
Show a scatterplot using two informative variables to distinguish small cars from other cars.
    
Which cars are hard to distinguish from small cars?
</div>

**Answer:**

- Since small cars (red points) are the leftmost points on the PC1 axis, I can say that small cars have less power (smaller engine size and less tank capacity), since pc1 is most dominated by the 3 variables ['Weight', 'Engine', 'Tank'].

- Since small cars are more vertically gathered, compressed, and centered near the zero-line of the PC2 axis, and since PC2 is most dominated by 3 variables: ['RPM', 'MaxPrice', and 'MidPrice'], I can tell that small cars are the cheapest in general. And there is no huge gap in their prices, since their distance from the zero-line is smaller compared to other types of cars like Midsize cars (green) for instance, which tend to be very sparse and have large vertical variance.

In [266]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Div, Column
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10
from bokeh.layouts import column

factors = sorted(cars['Type'].unique())
palette = Category10[len(factors)]

x, y = 'Weight', 'MidPrice'

# Create scatterplot: Weight vs Price
p = figure(title=f"{x} vs {y}: Distinguishing Small Cars", width=600, height=600)
p.scatter(x=x, y=y, source=cars, size=9, legend_field='Type',
          color=factor_cmap('Type', palette=palette, factors=factors))

# Add axis labels
p.xaxis.axis_label = x
p.yaxis.axis_label = y

# Customize legend
p.legend.title = "Car Type"
p.legend.location = "top_left"

# Create the caption
caption = Div(text=f"<p><strong>Figure:</strong> This plot visualizes the relationship between car {x} and {y}, categorized by car type. "
                   "We can clearly notice that small cars are in general the lightest and the cheapest.</p>",
              width=600)

# Arrange the plot and caption together
layout = column(p, caption)

show(layout)



<div class="alert alert-success">

**Distinguish large cars and vans**

Show a scatterplot that makes it easy to distinguish between large cars and vans. Which two variables do you pick?
</div>

In [267]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Div, Column
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10
from bokeh.layouts import column


x, y = 'Engine', 'Tank'

# Create scatterplot: Weight vs Price
p = figure(title=f"{x} vs {y}: Distinguishing Large Cars and Vans", width=600, height=600)
p.scatter(x=x, y=y, source=cars, size=9, legend_field='Type',
          color=factor_cmap('Type', palette=palette, factors=factors))

# Add axis labels
p.xaxis.axis_label = x
p.yaxis.axis_label = y

# Customize legend
p.legend.title = "Car Type"
p.legend.location = "top_left"

# Create the caption
caption = Div(text=f"<p><strong>Figure:</strong> This plot visualizes the relationship between {x} and {y}, categorized by car type. "
                   "Lerge cars have larger engine size than Vans</p>",
              width=600)

# Arrange the plot and caption together
layout = column(p, caption)

show(layout)

